In [3]:
from fuzzywuzzy import fuzz
import pandas as pd
import itertools
import numpy as np
import matplotlib.pyplot as plt
import time
import re
import string
from scipy import stats
import boto3
import awswrangler as wr
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
import json
import os
import boto3
from unidecode import unidecode
from dotenv import load_dotenv

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)

In [4]:
with open('/home/grs000362/.aws/credentials') as file_cred:
    file = file_cred.read()
    
with open('./.env','w') as file_env:
    file_env.write(file)
    
load_dotenv(override=True)
    
ATHENA_USER  = os.environ['aws_access_key_id']
ATHENA_PASSWORD = os.environ['aws_secret_access_key']
aws_session_token = os.environ['aws_session_token'] 


def import_data_athena(name):
    sql_file = name + '.sql'

    my_session = boto3.session.Session(
    aws_access_key_id = ATHENA_USER,
    aws_secret_access_key = ATHENA_PASSWORD,
    aws_session_token = aws_session_token,
    region_name = 'us-east-1')
    
    with open('./sql/'+sql_file,'r') as sql_file:
        sql_query = sql_file.read()

    df = wr.athena.read_sql_query(sql_query, database="ml_data_integration",ctas_approach=False,boto3_session=my_session)
    df = df.drop_duplicates()
    df = df.reset_index(drop = True)
    df = df.loc[:,~df.columns.duplicated()]
        
    return df

In [5]:
df_address = import_data_athena('address')

In [8]:
df_address['key'] = df_address['person_id'].astype(str) + df_address['zipcode'].astype(str) 

In [9]:
def bvs_address(json_,key = None):
    
    json_place =  json_['localizacao']
    cep    = json_place['cep']
    city   = json_place['cidade']
    state  = json_place['unidadeFederativa']
    neighborhood = json_place['bairro']
    number =  json_place['numeroLogradouro']
    street = json_place['nomeLogradouro']
    tipoLogradouro = json_place['tipoLogradouro'] 
    
    
    dic = {'key':[key],
           'tipoLogradouro':[tipoLogradouro],
           'bvs_cep':[cep],
           'bvs_city':[city],
           'bvs_state':[state],
           'bvs_street':[street],
           'bvs_neighborhood':[neighborhood],
           'bvs_number':[number]}
    
    return dic  

In [10]:
key_array = df_address['key'].values

df_final = pd.DataFrame({'key':[],'tipoLogradouro':[],'bvs_cep':[],'bvs_city':[],'bvs_state':[],'bvs_street':[],'bvs_neighborhood':[],'bvs_number':[]})

for k in key_array:
    
    temp_df  =  df_address[df_address['key'] == k]
    
    temp_df = temp_df.reset_index()
    json_temp =  json.loads(temp_df['value'][0])
    
    dic_temp = bvs_address(json_ = json_temp,key = k)
    
    df_temp_dic = pd.DataFrame(dic_temp)
    
    df_final = df_final.append(df_temp_dic)

In [12]:
df_final['bvs_street'] = df_final['tipoLogradouro'].astype(str) +' '+df_final['bvs_street'].astype(str)

In [13]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]:#@+-*')

def clean_text(text):
    
    text = text.lower() 
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)        
    text = unidecode(text)
    return text

In [14]:
df_final_address = df_final.merge(df_address, on = 'key')

In [15]:
df_final_address = df_final_address[df_final_address['bvs_city'].notnull()]
df_final_address = df_final_address[df_final_address['bvs_street'].notnull()]
df_final_address = df_final_address[df_final_address['streetaddress'].notnull()]
df_final_address = df_final_address[df_final_address['neighborhood'].notnull()]
df_final_address = df_final_address[df_final_address['city'].notnull()]
df_final_address = df_final_address[df_final_address['streetaddress'].notnull()]
df_final_address = df_final_address[df_final_address['neighborhood'].notnull()]

In [16]:
prep_string = ['bvs_city','bvs_street','bvs_neighborhood','city','streetaddress','neighborhood']

for string_ in prep_string:
    
    df_final_address[string_] = df_final_address[string_].apply(clean_text)

# Text Similarity

## Levenshtein Distance

In [17]:
df_final_address['levenshtein_distance_city'] = df_final_address.apply(lambda row: fuzz.ratio(row['bvs_city'],row['city']),axis = 1) 
df_final_address['levenshtein_distance_street'] = df_final_address.apply(lambda row: fuzz.ratio(row['bvs_street'],row['streetaddress']),axis = 1)
df_final_address['levenshtein_distance_neighborhood'] = df_final_address.apply(lambda row: fuzz.ratio(row['bvs_neighborhood'],row['neighborhood']),axis = 1) 

## Cosine Similarity

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


def string_cosine_similarity(d1,d2):
    count_vect = CountVectorizer()
    


    if len(d1) <= 3 or len(d2)<=3:
        
        
        d1 = d1 + ' ' + d1 + d1 
        d2 = d2 + ' ' + d2 + d2
        
        corpus = [d1,d2]
        
        X_train_counts = count_vect.fit_transform(corpus)
        cosine_sim = cosine_similarity(X_train_counts.toarray())
        cosine_sim = cosine_sim[1,0]
        
        
    else:
        
        corpus = [d1,d2]
        X_train_counts = count_vect.fit_transform(corpus)
        cosine_sim = cosine_similarity(X_train_counts.toarray())
        cosine_sim = cosine_sim[1,0]
    
    return cosine_sim
  

In [19]:
df_final_address = df_final_address[df_final_address['bvs_street'] != '']
df_final_address = df_final_address[df_final_address['bvs_neighborhood'] != '']
df_final_address = df_final_address[df_final_address['bvs_city'] != '']

df_final_address = df_final_address[df_final_address['city'] != '']
df_final_address = df_final_address[df_final_address['streetaddress'] != '']
df_final_address = df_final_address[df_final_address['neighborhood'] != '']

In [20]:
df_final_address['cosine_similarity_city'] = df_final_address.apply(lambda row: string_cosine_similarity(d1 = row['bvs_city'],d2 = row['city']),axis = 1) 
df_final_address['cosine_similarity_street'] = df_final_address.apply(lambda row: string_cosine_similarity(d1 = row['bvs_street'],d2 = row['streetaddress']),axis = 1) 
df_final_address['cosine_similarity_neighborhood'] = df_final_address.apply(lambda row: string_cosine_similarity(d1 = row['bvs_neighborhood'],d2 = row['neighborhood']),axis = 1) 

In [21]:
df_final_address['levenshtein_distance_city']   = df_final_address['levenshtein_distance_city'] / 100
df_final_address['levenshtein_distance_street'] = df_final_address['levenshtein_distance_street'] / 100
df_final_address['levenshtein_distance_neighborhood'] =  df_final_address['levenshtein_distance_neighborhood'] / 100

In [22]:
df_final_address

,key,tipoLogradouro,bvs_cep,bvs_city,bvs_state,bvs_street,bvs_neighborhood,bvs_number,person_id,city,state,number,neighborhood,streetaddress,zipcode,value,levenshtein_distance_city,levenshtein_distance_street,levenshtein_distance_neighborhood,cosine_similarity_city,cosine_similarity_street,cosine_similarity_neighborhood
0,1516863446430000,R,46430000,guanambi,BA,r antonio teixeira filho,araujo,53,15168634,guanambi,BA,53,araujo,rua antonio teixeira firmo,46430000,"{""rev"":3,""objType"":""BoaVistaAcertaPositivoPers...",1.00,0.88,1.00,1.000000,0.577350,1.0
1,314246368682000,,68680000,tome acu,PA,cameta,quatro bocas,64,3142463,quatro bocas,PA,93,tsuruzaki,rua orlando silva,68682000,"{""rev"":1,""objType"":""BoaVistaAcertaPositivoPers...",0.40,0.25,0.29,0.000000,0.000000,0.0
2,157890114439080,R,04439070,sao paulo,SP,r eng joao lang,jd martini,50,15789011,sao paulo,SP,15,jardim martini,rua jose moreira dos santos,4439080,"{""rev"":1,""objType"":""BoaVistaAcertaPositivoPers...",1.00,0.48,0.83,1.000000,0.000000,0.5
3,1281166713053281,R,13053151,campinas,SP,r jeronimo mendonca,jd campo belo,,12811667,campinas,SP,62,dom gilberto,antonio scavone,13053281,"{""rev"":3,""objType"":""BoaVistaAcertaPositivoPers...",1.00,0.41,0.48,1.000000,0.000000,0.0
4,1281166713053281,R,13053151,campinas,SP,r jeronimo mendonca,jd campo belo,,12811667,campinas,SP,62,jardim dom gilberto,rua antonio scavone,13053281,"{""rev"":3,""objType"":""BoaVistaAcertaPositivoPers...",1.00,0.47,0.50,1.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61007,406980920210910,AV,25585000,sao joao de meriti,RJ,av estacio de sa,parque novo rio,5,4069809,rio de janeiro,RJ,Bloco paraia apt 503,santo cristo,rua da america 81,20210910,"{""rev"":4,""objType"":""BoaVistaAcertaPositivoPers...",0.44,0.36,0.52,0.288675,0.000000,0.0
61008,326809422051002,R,20550035,rio de janeiro,RJ,r alzira brandao,tijuca,87,3268094,rio de janeiro,RJ,582,copacabana,rua barata ribeiro,22051002,"{""rev"":1,""objType"":""BoaVistaAcertaPositivoPers...",1.00,0.53,0.25,1.000000,0.000000,0.0
61009,165489288132150,R,88131440,palhoca,SC,r carlos weingartner,centro,158,1654892,palhoca,SC,120,pagani,rua najla carone guedert,88132150,"{""rev"":2,""objType"":""BoaVistaAcertaPositivoPers...",1.00,0.45,0.17,1.000000,0.000000,0.0
61010,1563666323895740,R,23895740,seropedica,RJ,r antonio rosa,santa sofia,24,15636663,seropedica,RJ,24,santa sofia,rua antonio rosa,23895740,"{""rev"":1,""objType"":""BoaVistaAcertaPositivoPers...",1.00,0.93,1.00,1.000000,0.816497,1.0


# Kmeans

In [23]:
from sklearn.cluster import KMeans

In [24]:

kmeans_features = ['levenshtein_distance_city',
                   'levenshtein_distance_street',
                   'levenshtein_distance_neighborhood',
                   'cosine_similarity_city',
                   'cosine_similarity_street',
                   'cosine_similarity_neighborhood']
X = df_final_address[kmeans_features].values
kmeans = KMeans(n_clusters=3, random_state=0).fit(X)

In [25]:
df_final_address['cluster'] = kmeans.labels_

In [26]:
df_final_address.groupby('cluster').count()['key']

cluster
0    26883
1    20388
2    12031
Name: key, dtype: int64

In [27]:
df_final_address.groupby('cluster').count()['key'] / len(df_final_address)

cluster
0    0.453324
1    0.343800
2    0.202877
Name: key, dtype: float64